In [ ]:
!nvidia-smi -L

In [ ]:
# @markdown # **Step 0.** _Removing sample_data_

!rm -rf /content/sample_data

In [ ]:
# @markdown # **Step 1.** _Installing repositories_

import os

from google.colab import output

%cd /content

# Cloning a repository (https://github.com/AnyGogin31/InvokeAI-Colab)

if not os.path.exists("./InvokeAI-Colab"):
    !git clone -q https://github.com/AnyGogin31/InvokeAI-Colab.git

# Cloning a repository (https://github.com/invoke-ai/InvokeAI)

if not os.path.exists("./InvokeAI"):
    !git clone -q https://github.com/invoke-ai/InvokeAI.git

# Clearing the output

output.clear()

In [ ]:
# @markdown # **Step 2.** _Installing conda_

import os

from google.colab import output

%cd /content

# Installing conda

if not os.path.exists("/usr/local/bin/conda"):
    !wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
    !rm Miniconda3-latest-Linux-x86_64.sh

# Clearing the output

output.clear()

In [ ]:
# @markdown # **Step 3.** _Installing dependencies_

import os

from google.colab import output

%cd /content

# Setting environment variables

invokeai_root_dir = os.path.join(os.getcwd(), "invokeai")

os.environ["INVOKEAI_ROOT_DIR"] = invokeai_root_dir

kernel_name = "invokeai_kernel"
kernel_prefix = os.path.join("/usr/local/envs/", kernel_name)

os.environ["INVOKEAI_KERNEL_NAME"] = kernel_name
os.environ["INVOKEAI_KERNEL_PREFIX"] = kernel_prefix

# Installing dependencies

%cd ./InvokeAI-Colab

!chmod +x *.sh
!bash install_dependencies.sh

# Clearing the output

output.clear()

In [ ]:
# @markdown # Loading from google drive (optional)

import os
import shutil

from google.colab import drive, output

drive_path = "/content/drive"
drive_invokeai_root_dir = f"{drive_path}/MyDrive/InvokeAI/invokeai"

try:
    drive.mount(drive_path, force_remount=False)
    os.makedirs(drive_invokeai_root_dir, exist_ok=True)
except:
    print("Error connecting storage")

shutil.copytree(drive_invokeai_root_dir, invokeai_root_dir)
output.clear()

In [ ]:
# @markdown # **Step 4.** _Launch of InvokeAI_

import os

from google.colab import userdata

%cd /content/InvokeAI-Colab

# Launch Configuration

config_path = os.path.join(invokeai_root_dir, "invokeai.yaml")

os.environ["INVOKEAI_CONFIG_PATH"] = config_path

os.environ["NGROK_TOKEN"] = userdata.get("NGROK_TOKEN")
os.environ["NGROK_DOMAIN"] = userdata.get("NGROK_DOMAIN")

# Launch of InvokeAI

!chmod +x *.sh
!bash start.sh

In [ ]:
# @markdown # Upload to google drive (optional)

import os
import shutil
import filecmp

from google.colab import drive, output

drive_path = "/content/drive"
drive_invokeai_root_dir = f"{drive_path}/MyDrive/InvokeAI/invokeai"

try:
    drive.mount(drive_path, force_remount=False)
    os.makedirs(drive_invokeai_root_dir, exist_ok=True)
except:
    print("Error connecting storage")


def synchronize_directories(src_dir, dst_dir, exclude_dirs=None):
    if exclude_dirs is None:
        exclude_dirs = {}

    for item_name in os.listdir(src_dir):
        src_path = os.path.join(src_dir, item_name)
        dst_path = os.path.join(dst_dir, item_name)

        if any(src_path.startswith(os.path.join(src_dir, exclude)) for exclude in exclude_dirs if exclude_dirs[exclude]):
            print(f"Skipped: {src_path}")
            continue

        if os.path.isdir(src_path):
            if os.path.exists(dst_path) and not os.path.isdir(dst_path):
                os.remove(dst_path)
                print(f"Deleted file: {dst_path}")

            os.makedirs(dst_path, exist_ok=True)
            synchronize_directories(src_path, dst_path, exclude_dirs)
        else:
            if os.path.exists(dst_path) and os.path.isdir(dst_path):
                shutil.rmtree(dst_path)
                print(f"Deleted directory: {dst_path}")

            if not os.path.exists(dst_path) or not filecmp.cmp(src_path, dst_path, shallow=False):
                shutil.copy2(src_path, dst_path)
                print(f"Copied/Updated: {dst_path}")

    for item_name in os.listdir(dst_dir):
        dst_path = os.path.join(dst_dir, item_name)
        src_path = os.path.join(src_dir, item_name)

        if not os.path.exists(src_path):
            if os.path.isdir(dst_path):
                shutil.rmtree(dst_path)
                print(f"Deleted directory: {dst_path}")
            else:
                os.remove(dst_path)
                print(f"Deleted file: {dst_path}")

exclude_flux_models = True # @param {type: "boolean"}
exclude_t5_encoder = True # @param {type: "boolean"}
exclude_clip_embed = True # @param {type: "boolean"}

exclude_list = {
    "flux": exclude_flux_models,
    "t5_encoder": exclude_t5_encoder,
    "clip_embed": exclude_clip_embed
}

synchronize_directories(invokeai_root_dir, drive_invokeai_root_dir, exclude_list)